In [1]:
import pickle
from statistics import mean 
import collections

In [2]:
data = 'heloc'
method = 'dser'

In [3]:
from sklearn.preprocessing import MinMaxScaler

# function to scale the values
def scale_values(input_list, feature_range=(0, 1)):
    # Reshape the input list to a 2D array for MinMaxScaler
    values_2d = [[value] for value in input_list]

    # Initialize the MinMaxScaler with the specified feature range
    scaler = MinMaxScaler(feature_range=feature_range)

    # Fit and transform the data using the scaler
    scaled_values = scaler.fit_transform(values_2d)

    # Extract the scaled values from the 2D array and return as a list
    scaled_list = [scaled_value[0] for scaled_value in scaled_values]

    return scaled_list

In [4]:
with open(data+'_'+method+'.pickle', 'rb') as f:
    results = pickle.load(f)

In [5]:
len(results)

5

In [6]:
#print(results[0])

In [7]:
sf_query = []
sf_nh_knn = []
sf_nun_knn = []
sf_nun = []
mahalanobis = []
sparsity = []
ood_distance = []
trust_score = []
lipschitz = []

In [8]:
for item in results:
    if len(item)>0:
        sf_query.append(item['sf_query'])
        sf_nun.append(item['sf_nun'])
        mahalanobis.append(item['mahalanobis'])
        sparsity.append(item['sparsity'])
        ood_distance.append(item['ood_distance'])
        trust_score.append(item['trust_score'])
        lipschitz.append(item['lipschitz'])

In [9]:
sf_query = [x for xs in sf_query for x in xs]
sf_nun = [x for xs in sf_nun for x in xs]
mahalanobis = [x for xs in mahalanobis for x in xs]
sparsity = [x for xs in sparsity for x in xs]
ood_distance = [x for xs in ood_distance for x in xs]
trust_score = [x for xs in trust_score for x in xs]
trust_score_sc = scale_values(trust_score)
lipschitz = [x for xs in lipschitz for x in xs]
lipschitz_sc = scale_values(lipschitz)

In [10]:
metric_zip = zip(sf_query, sf_nun, sparsity, ood_distance, trust_score_sc, lipschitz_sc)

In [11]:
unitary = []

# create unitary metric by combining the metrics in a relevant way
for sf_q, sf_n, sp, ood, ts, lips in metric_zip:
    
    if (sp == 0):
        val = sf_q + 0 + ts + (1-ood) + (1-lips)
    else:
        val = sf_q + (1/sp) + ts +(1-ood) + (1-lips)
    
    unitary.append(val)

In [12]:
unitary = mean(unitary)
unitary

2.8681457751510533

In [13]:
sf_query = mean(sf_query)
sf_query

0.03858162361197136

In [66]:
sf_nun = mean(sf_nun)
sf_nun

0.29842311525404774

In [67]:
mahalanobis = mean(mahalanobis)
mahalanobis

4.682933161177572

In [68]:
ood_distance = mean(ood_distance)
ood_distance

0.26358755586347193

In [69]:
trust_score = mean(trust_score)
trust_score

1.0063791606694368

In [70]:
trust_score_sc = mean(trust_score_sc)
trust_score_sc

0.5457876166090099

In [71]:
lipschitz = mean(lipschitz)
lipschitz

1.1436068061841331

In [72]:
lipschitz_sc = mean(lipschitz_sc)
lipschitz_sc

0.4001933804708239

In [73]:
sp = sparsity.copy()
sp = [1/val for val in sp]
mean(sp)

0.9475574712643678

In [38]:
sparse = collections.Counter(sparsity)
sparse

Counter({1: 623, 2: 73})

In [39]:
new_dict = {}
three = 0
one = 0
two = 0
for key in sparse:
    if key>=3:
        three += sparse[key]
    elif key == 1:
        one = sparse[key]
    elif key == 2:
        two = sparse[key]

new_dict['1'] = one
new_dict['2'] = two
new_dict['3+'] = three

new_dict

{'1': 623, '2': 73, '3+': 0}

In [40]:
new_dict_p = {}

new_dict_p['1'] = (new_dict['1']/(new_dict['1']+new_dict['2']+new_dict['3+']))*100
new_dict_p['2'] = (new_dict['2']/(new_dict['1']+new_dict['2']+new_dict['3+']))*100
new_dict_p['3+'] = (new_dict['3+']/(new_dict['1']+new_dict['2']+new_dict['3+']))*100

new_dict_p

{'1': 89.51149425287356, '2': 10.488505747126437, '3+': 0.0}